# Imports

In [104]:
import pandas as pd
import numpy as np
import json
import datetime
import ast
import dtale

from sklearn import preprocessing as pp
import pickle

from sklearn import model_selection as ms
import category_encoders as ce
# from dataprep.datasets  import load_dataset
# from dataprep.eda       import create_report

#from boruta   import BorutaPy
from lightgbm import LGBMClassifier

from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBRegressor
from sklearn.linear_model    import LogisticRegression, SGDRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, BaggingRegressor
from sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMRegressor

# from category_encoders import TargetEncoder, CountEncoder as ce

## Helper Functions

In [105]:
def rescalling (X_test):
    
    mm = pickle.load(open('parameters/mm_avg_delivery_time_days.pkl','rb'))
    X_test['avg_delivery_time_days'] = mm.transform(X_test[['avg_delivery_time_days']])
    
    rs_ar = pickle.load(open('parameters/rs_average_rating.pkl','rb'))
    X_test['average_rating'] = rs_ar.transform(X_test[['average_rating']])
    
    rs_nr = pickle.load(open('parameters/rs_number_of_reviews.pkl','rb'))
    X_test['number_of_reviews'] = rs_nr.transform(X_test[['number_of_reviews']])
    
    return X_test

def transform (X_test):
    
    for att in ['brand', 'product', 'seller']:
        fe = pickle.load(open(f'parameters/fe_{att}.pkl', 'rb'))
        X_test = fe.transform(X_test)
    
    return X_test


def transf_cat(X_test):

    le_suitable = pickle.load(open('parameters/le_suitable.pkl','rb'))
    X_test['Suitable For'] = le_suitable.transform(X_test['Suitable For'])

    le_reversible = pickle.load(open('parameters/le_reversible.pkl','rb'))
    X_test['Reversible'] = le_reversible.transform(X_test['Reversible'])

    le_ideal = pickle.load(open('parameters/le_ideal.pkl','rb'))
    X_test['Ideal For'] = le_ideal.transform(X_test['Ideal For'])

    le_out = pickle.load(open('parameters/le_out.pkl','rb'))
    X_test['out_of_stock'] = le_out.transform(X_test['out_of_stock'])

    return X_test


def target(X_test, y_test):
    encoder_style_code = pickle.load(open('parameters/encoder_style_code.pkl','rb'))
    X_test['Style Code'] = encoder_style_code.transform(X_test['Style Code'], y_test) 

    encoder_pack_of = pickle.load(open('parameters/encoder_pack_of.pkl','rb'))
    X_test['Pack of'] = encoder_pack_of.transform(X_test['Pack of'], y_test) 

    return X_test

def test_preparation(X_test, y_test):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    X_test = transf_cat(X_test)
    X_test = target(X_test, y_test)
    y_test = log_var_resposta(y_test)
    
    return X_test, y_test


def log_var_resposta(df):
    df = np.log1p(df)
    return df


def feature_selection(df):
    features_selected = ['average_rating',
    'number_of_reviews',
    'brand',
    # 'category',
    # 'crawled_at',
    'out_of_stock',
    'avg_delivery_time_days',
    #  'product_details',
    'seller',
    #  'sub_category',
    #  'fabrication_time',
    #  'title',
    #  'Fabric',
    #  'Pattern',
    'Style Code',
    'Pack of',
    #  'Type',
    #  'Sleeve',
    #  'Fit',
    'Ideal For',
    'Suitable For',
    'Reversible',
    #  'Fabric Care',
    'product']
    
    return df[features_selected]


In [106]:
# Funções para limpeza dos dados de teste

def limpeza (df):
    df = df.drop(columns=['_id', 'pid','description', 'images'])
    # transformando de string para data
    df['crawled_at'] = pd.to_datetime(df['crawled_at']).dt.date

    return df

def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    return df

def submission_preparation(df, features_selected):
    
    df = limpeza(df)
    df = feature_engen(df)
    df = rescalling(df)
    df = transform(df)
    
    return df[features_selected]


In [107]:
# ????

def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []
    
def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

# Load Data

In [108]:
# Opening JSON file
train = open('dataset/raw/train.json')
 
# returns JSON object as a dictionary
data_train_aux = json.load(train)

data_train = pd.json_normalize(data_train_aux, record_path = 'data')
data_train.columns = data_train_aux['columns']
data_train.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10 21:17:28,Navy Blue Printed Boxers Has An Inner Elasti...,['https://rukminim1.flixcart.com/image/128/128...,False,8,BXRFTZF7JGX75DAW,"[{'Color': 'Dark Blue'}, {'Fabric': 'Pure Cott...",ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11 01:02:46,smiley printed tshirt on round neck cotton tshirt,['https://rukminim1.flixcart.com/image/128/128...,False,12,TSHFWQM96UHR6A4Q,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11 00:43:37,Free Authority Presents this Crew Neck Yellow ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,SWSFWCXH2WF6ZYRB,"[{'Color': 'Yellow'}, {'Fabric': 'Polycotton'}...",BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10 21:22:10,Cotton Blend FabricCollar / Polo Neck White & ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,TSHFHQH3HKDAGGK9,"[{'Type': 'Polo Neck'}, {'Sleeve': 'Short Slee...",HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10 23:36:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,5,TSHFW9CJZSYUU6UX,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


# Data Description

In [109]:
# retirar descrição
# retirar imagem
# verificar ids duplicados em id produto
# Verificar outliers no price

## Removendo colunas

In [110]:
data_train = data_train.drop(columns=['description', 'images'])

## Transformando tipo de dados

In [111]:
data_train.dtypes

_id                        object
average_rating            float64
number_of_reviews           int64
brand                      object
category                   object
crawled_at                 object
out_of_stock                 bool
avg_delivery_time_days      int64
pid                        object
product_details            object
seller                     object
sub_category               object
fabrication_time            int64
title                      object
actual_price              float64
dtype: object

In [112]:
# transformando de string para data
data_train['crawled_at'] = pd.to_datetime(data_train['crawled_at']).dt.date

## Tratando valores nulos

In [113]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.149985
dtype: float64

In [114]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.149985
dtype: float64

In [115]:
## Tratando variaveis respostas nulas
prices = data_train[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()

precos_nulos = data_train.loc[data_train['actual_price'].isna(), ['_id','pid']]

precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

data_train_aux = pd.merge(data_train, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
data_train['actual_price'] = data_train_aux['actual_price_x'].fillna(0) + data_train_aux['actual_price_y'].fillna(0)

data_train = data_train.dropna(subset=['actual_price'])

## Entendendo product Details

In [116]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23309 entries, 0 to 23308
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     23309 non-null  object 
 1   average_rating          23309 non-null  float64
 2   number_of_reviews       23309 non-null  int64  
 3   brand                   23309 non-null  object 
 4   category                23309 non-null  object 
 5   crawled_at              23309 non-null  object 
 6   out_of_stock            23309 non-null  bool   
 7   avg_delivery_time_days  23309 non-null  int64  
 8   pid                     23309 non-null  object 
 9   product_details         23309 non-null  object 
 10  seller                  23309 non-null  object 
 11  sub_category            23309 non-null  object 
 12  fabrication_time        23309 non-null  int64  
 13  title                   23309 non-null  object 
 14  actual_price            23309 non-null

In [117]:
# def is_valid_dict(x):
#     return isinstance(x, dict)

# invalid_dict_rows = data_train[~data_train['product_details'].apply(is_valid_dict)]
# print(invalid_dict_rows)


In [118]:
# # Função para analisar strings JSON
# def parse_json(x):
#     try:
#         return json.loads(x)
#     except Exception as e:
#         print(e)
#         return {}

# # Aplicar a função de análise a cada linha da coluna 'product_details'
# data_train['product_details'] = data_train['product_details'].apply(parse_json)

# # Converter o dicionário em várias colunas
# df_details = data_train['product_details'].apply(pd.Series)

# # Verifique o DataFrame resultante
# print(df_details.head())

In [119]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []



def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

list = data_train['product_details'].apply(lambda x: f(x))
df_details = pd.DataFrame([junta_dict(row) for row in list])



In [120]:
df_details.head()

,Color,Fabric,Pattern,Sales Package,Style Code,Pack of,Type,Sleeve,Fit,Neck Type,...,Foot Coverage,Bust in inch,Shoulder in inch,Sleeve in inch,Pleated,Design,Weave type,Fabric care,Width,Height
0,Dark Blue,Pure Cotton,Printed,1 boxer,EIBXCO053,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Cotton Blend,Printed,pack of 1,orange,1,Round Neck,Short Sleeve,Regular,Round Neck,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yellow,Polycotton,Graphic Print,1 Terry Cotton Sweatshirt for Men,STY-20-21-001595,NaN,NaN,Full Sleeve,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Cotton Blend,Solid,2 pc T-shirts,HU2002PO-HS-IB-WH,2,Polo Neck,Short Sleeve,Regular,Polo Neck,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Cotton Blend,Printed,NaN,RC021_White,1,Round Neck,Short Sleeve,Regular,Round Neck,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
df_details.columns.tolist()

['Color',
 'Fabric',
 'Pattern',
 'Sales Package',
 'Style Code',
 'Pack of',
 'Type',
 'Sleeve',
 'Fit',
 'Neck Type',
 'Ideal For',
 'Size',
 'Suitable For',
 'Sleeve Type',
 'Reversible',
 'Fabric Care',
 'Brand Color',
 'Neck',
 'Occasion',
 'Hooded',
 'Other Details',
 'Brand Fit',
 'Generic Name',
 'Country of Origin',
 'Model Name',
 'Number of Contents in Sales Package',
 '',
 'Collar',
 'Pack Of',
 'Faded',
 'Rise',
 'Distressed',
 'Closure',
 'Pockets',
 'Hem',
 'Secondary Color',
 'Bottom Length',
 'Dupatta Length',
 'Top Length',
 'Stitching Type',
 ' ',
 'Waistband',
 'Series',
 'Warranty Service Type',
 'Warranty Summary',
 'Covered in Warranty',
 'Not Covered in Warranty',
 'Domestic Warranty',
 'Fly',
 'Other Dimensions',
 'Top Fabric',
 'Bottom Fabric',
 'Top Type',
 'Bottom Type',
 'Length Type',
 'Package contains',
 'Care instructions',
 'Sole Material',
 'Inner Material',
 'Outer Material',
 'Weight',
 'Care Instructions',
 'Alteration Required',
 'Belt Loops',
 'P

In [122]:
df_drop_columns = df_details.drop(columns=['', ' '])

# Removendo valores faltantes acima de 50%
limite_nulos = len(df_details) * 0.5  
df_drop = df_details.dropna(thresh = limite_nulos, axis=1)
df_new = df_drop.copy()

In [123]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23309 entries, 0 to 23308
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Fabric        21576 non-null  object
 1   Pattern       21361 non-null  object
 2   Style Code    21370 non-null  object
 3   Pack of       15131 non-null  object
 4   Type          14570 non-null  object
 5   Sleeve        15858 non-null  object
 6   Fit           14097 non-null  object
 7   Ideal For     14342 non-null  object
 8   Suitable For  16141 non-null  object
 9   Reversible    12201 non-null  object
 10  Fabric Care   18294 non-null  object
dtypes: object(11)
memory usage: 2.0+ MB


In [124]:
# Trocar os NaN por 'outros'

df_new.loc[df_new['Fabric'].isnull(), 'Fabric'] = 'outros'
df_new.loc[df_new['Pattern'].isnull(), 'Pattern'] = 'outros'
df_new.loc[df_new['Style Code'].isnull(), 'Style Code'] = 'outros'
df_new.loc[df_new['Pack of'].isnull(), 'Pack of'] = 'outros'
df_new.loc[df_new['Type'].isnull(), 'Type'] = 'outros'
df_new.loc[df_new['Sleeve'].isnull(), 'Sleeve'] = 'outros'
df_new.loc[df_new['Fit'].isnull(), 'Fit'] = 'outros'
df_new.loc[df_new['Ideal For'].isnull(), 'Ideal For'] = 'outros'
df_new.loc[df_new['Suitable For'].isnull(), 'Suitable For'] = 'outros'
df_new.loc[df_new['Reversible'].isnull(), 'Reversible'] = 'outros'
df_new.loc[df_new['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'

df_new.head()


,Fabric,Pattern,Style Code,Pack of,Type,Sleeve,Fit,Ideal For,Suitable For,Reversible,Fabric Care
0,Pure Cotton,Printed,EIBXCO053,1,outros,outros,outros,outros,outros,outros,outros
1,Cotton Blend,Printed,orange,1,Round Neck,Short Sleeve,Regular,Men,Western Wear,No,Do not Iron on print/embroidery/embellishment
2,Polycotton,Graphic Print,STY-20-21-001595,outros,outros,Full Sleeve,outros,outros,Western Wear,No,"Hand Wash, Reverse and Dry"
3,Cotton Blend,Solid,HU2002PO-HS-IB-WH,2,Polo Neck,Short Sleeve,Regular,Men,Western Wear,No,Gentle Machine Wash
4,Cotton Blend,Printed,RC021_White,1,Round Neck,Short Sleeve,Regular,Men,Western Wear,outros,Regular Machine Wash


In [125]:
# # Suponha que você tenha uma coluna chamada 'Fabric' com valores do tipo object
# df_new['Fabric'] = df_new['Fabric'].astype(str)
# df_new['Pattern'] = df_new['Pattern'].astype(str)
# df_new['Style Code'] = df_new['Style Code'].astype(str)
# df_new['Pack of'] = df_new['Pack of'].astype(str)
# df_new['Type'] = df_new['Type'].astype(str)
# df_new['Sleeve'] = df_new['Sleeve'].astype(str)
# df_new['Fit'] = df_new['Fit'].astype(str)
# df_new['Ideal For'] = df_new['Ideal For'].astype(str)
# df_new['Suitable For'] = df_new['Suitable For'].astype(str)
# df_new['Reversible'] = df_new['Reversible'].astype(str)
# df_new['Fabric Care'] = df_new['Fabric Care'].astype(str)


# # Agora você pode usar o método str.split para dividir os valores
# df_new['Fabric'] = df_new['Fabric'].str.split(',')
# df_new['Pattern'] = df_new['Pattern'].str.split(',')
# df_new['Style Code'] = df_new['Style Code'].str.split(',')
# df_new['Pack of'] = df_new['Pack of'].str.split(',')
# df_new['Type'] = df_new['Type'].str.split(',')
# df_new['Sleeve'] = df_new['Sleeve'].str.split(',')
# df_new['Fit'] = df_new['Fit'].str.split(',')
# df_new['Ideal For'] = df_new['Ideal For'].str.split(',')
# df_new['Suitable For'] = df_new['Suitable For'].str.split(',')
# df_new['Reversible'] = df_new['Reversible'].str.split(',')
# df_new['Fabric Care'] = df_new['Fabric Care'].str.split(',')


In [126]:
# Dados sem virgulas: Style Code, Pack of, Suitable For, Reversible, 'Ideal For'


# Ideal for: 'Men', 'Men, Boys'/ 'Boys, Men' = 'all_ages',
            # 'Boys, Girls, Men, Women' = 'unisex_all_ages', 'Women, Men' = 'Unisex'

def type_sex(value):
    if value == 'Men':
        return 'Men'
    elif value == 'Men, Boys':
        return 'all_ages'
    elif value == 'Boys, Men':
        return 'all_ages'
    elif value == 'Boys, Girls, Men, Women':
        return 'unisex_all_ages'
    elif value == 'Women, Men':
        return 'Unisex'
    else:
        return 'outros'

df_new['Ideal For'] = df_new['Ideal For'].map(type_sex)

In [127]:
df_junto = pd.concat([data_train, df_new], axis=1)


# Feature Engineering

In [128]:
df4 = df_junto.copy()

In [129]:
## out_of_stock - transformar para int 
df4['out_of_stock'] = df4['out_of_stock'].astype('int64')

# brand - substituir por outros
df4['brand'] = df4['brand'].apply(lambda x: x.lower())

# criando feature product
df4['product'] = df4[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)

# EDA

In [130]:
# # Assigning a reference to a running D-Tale process
# d = dtale.show(df4)

# # Accessing data associated with D-Tale process
# tmp = d.data.copy()
# tmp['d'] = 4

# # Altering data associated with D-Tale process
# # FYI: this will clear any front-end settings you have at the time for this process (filter, sorts, formatting)
# d.data = tmp

# # Shutting down D-Tale process
# #d.kill()

# # using Python's `webbrowser` package it will try and open your server's default browser to this process
# d.open_browser()

# # There is also some helpful metadata about the process
# d._data_id  # the process's data identifier
# d._url  # the url to access the process

# d2 = dtale.get_instance(d._data_id)  # returns a new reference to the instance running at that data_id

# dtale.instances()  # prints a list of all ids & urls of running D-Tale sessions

# Data Preparation

In [131]:
df5 = df4.copy()
df5.isna().sum()

_id                       0
average_rating            0
number_of_reviews         0
brand                     0
category                  0
crawled_at                0
out_of_stock              0
avg_delivery_time_days    0
pid                       0
product_details           0
seller                    0
sub_category              0
fabrication_time          0
title                     0
actual_price              0
Fabric                    0
Pattern                   0
Style Code                0
Pack of                   0
Type                      0
Sleeve                    0
Fit                       0
Ideal For                 0
Suitable For              0
Reversible                0
Fabric Care               0
product                   0
dtype: int64

## Split Dataset

In [132]:
X = df5.drop(['_id','pid','actual_price'], axis=1)
y = df5['actual_price']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.20)
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

## Rescalling

In [133]:
X_train.columns

Index(['average_rating', 'number_of_reviews', 'brand', 'category',
       'crawled_at', 'out_of_stock', 'avg_delivery_time_days',
       'product_details', 'seller', 'sub_category', 'fabrication_time',
       'title', 'Fabric', 'Pattern', 'Style Code', 'Pack of', 'Type', 'Sleeve',
       'Fit', 'Ideal For', 'Suitable For', 'Reversible', 'Fabric Care',
       'product'],
      dtype='object')

### Min Max Scaler

In [134]:
# 'avg_delivery_time_days',
mm = pp.MinMaxScaler()

X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])
pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))


### Robust Scaler

In [135]:
rs_average_rating = pp.RobustScaler()
rs_number_of_reviews = pp.RobustScaler()

X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])
pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))

##### Fazer Robust Scaler da variável resposta (??) #####


## Transformation

In [136]:
# Label encoder: Suitable For, Reversible, Ideal For, out_of_stock

le_suitable = pp.LabelEncoder()
le_reversible = pp.LabelEncoder()
le_ideal = pp.LabelEncoder()
le_out = pp.LabelEncoder()


X_train['Suitable For'] = le_suitable.fit_transform(X_train['Suitable For'])
pickle.dump(le_suitable, open('parameters/le_suitable.pkl', 'wb'))

X_train['Reversible'] = le_reversible.fit_transform(X_train['Reversible'])
pickle.dump(le_reversible, open('parameters/le_reversible.pkl', 'wb'))

X_train['Ideal For'] = le_ideal.fit_transform(X_train['Ideal For'])
pickle.dump(le_ideal, open('parameters/le_ideal.pkl', 'wb'))

X_train['out_of_stock'] = le_out.fit_transform(X_train['out_of_stock'])
pickle.dump(le_out, open('parameters/le_out.pkl', 'wb'))



In [137]:
# Instancie o codificador alvo para 'Style Code'
encoder_style_code = ce.TargetEncoder()
X_train['Style Code'] = encoder_style_code.fit_transform(X_train['Style Code'], y_train) 
pickle.dump(encoder_style_code, open('parameters/encoder_style_code.pkl', 'wb'))

# Instancie o codificador alvo para 'Pack of'
encoder_pack_of = ce.TargetEncoder()
X_train['Pack of'] = encoder_pack_of.fit_transform(X_train['Pack of'], y_train) 
pickle.dump(encoder_pack_of, open('parameters/encoder_pack_of.pkl', 'wb'))



### Frequency Encode

In [138]:
for att in ['brand', 'product', 'seller', 'category']:
    fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
    pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
    X_train = fe.transform(X_train)

### Response Variable

In [139]:
# Tranformando a variável resposta em logaritmica

y_train = np.log1p(y_train)

# Feature Selection

In [140]:
X_train.columns.tolist()

['average_rating',
 'number_of_reviews',
 'brand',
 'category',
 'crawled_at',
 'out_of_stock',
 'avg_delivery_time_days',
 'product_details',
 'seller',
 'sub_category',
 'fabrication_time',
 'title',
 'Fabric',
 'Pattern',
 'Style Code',
 'Pack of',
 'Type',
 'Sleeve',
 'Fit',
 'Ideal For',
 'Suitable For',
 'Reversible',
 'Fabric Care',
 'product']

In [141]:
features_selected = ['average_rating',
 'number_of_reviews',
 'brand',
 # 'category',
 # 'crawled_at',
 'out_of_stock',
 'avg_delivery_time_days',
#  'product_details',
 'seller',
#  'sub_category',
#  'fabrication_time',
#  'title',
#  'Fabric',
#  'Pattern',
 'Style Code',
 'Pack of',
#  'Type',
#  'Sleeve',
#  'Fit',
 'Ideal For',
 'Suitable For',
 'Reversible',
#  'Fabric Care',
 'product']



## Importancia das árvores

In [142]:
# Crie e treine o modelo LGBMClassifier
lgb_model = LGBMRegressor(n_jobs=-1, random_state=42)
X_train_sel = X_train[features_selected]
lgb_model.fit(X_train_sel, y_train)

# Obtenha as importâncias das características do modelo
importances = lgb_model.feature_importances_

# Ordene as características por importância decrescente
indices = np.argsort(importances)[::-1]

# Imprima o ranking das características
print('Feature ranking')
df = pd.DataFrame()

for i, j in zip(X_train_sel.columns, importances):
    aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])
    df = pd.concat([df, aux], axis=0)

print(df.sort_values('importance', ascending=False))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 609
[LightGBM] [Info] Number of data points in the train set: 18647, number of used features: 12
[LightGBM] [Info] Start training from score 6.155615
Feature ranking
                  feature  importance
0              Style Code        1132
0          average_rating         507
0                   brand         422
0                  seller         397
0                 product         225
0                 Pack of         136
0            Suitable For          58
0               Ideal For          57
0              Reversible          50
0  avg_delivery_time_days           9
0            out_of_stock           7
0       number_of_reviews           0


In [143]:
X_train[features_selected].head()

,average_rating,number_of_reviews,brand,out_of_stock,avg_delivery_time_days,seller,Style Code,Pack of,Ideal For,Suitable For,Reversible,product
18392,-0.7,-0.750,0.004183,0,0.2500,0.004183,1229.973876,1188.749877,0,3,2,0.553762
3891,-1.0,-1.125,0.013836,0,0.1875,0.013836,1268.061407,1188.749877,0,3,0,0.553762
14179,-1.2,-1.250,0.031855,0,0.1250,0.029871,1242.984723,1188.749877,3,3,0,0.553762
12351,-0.3,-0.375,0.017161,0,0.3750,0.016678,1243.114832,1188.749877,0,3,2,0.553762
18252,0.3,0.250,0.005792,0,0.5625,0.002521,1269.006418,1188.749877,0,3,0,0.553762


# Machine Learning

#### Test Preparation

In [144]:
X_test_mod, y_test_mod = test_preparation(X_test, y_test)

In [152]:
X_train[features_selected]

,average_rating,number_of_reviews,brand,out_of_stock,avg_delivery_time_days,seller,Style Code,Pack of,Ideal For,Suitable For,Reversible,product
18392,-0.7,-0.750,0.004183,0,0.2500,0.004183,1229.973876,1188.749877,0,3,2,0.553762
3891,-1.0,-1.125,0.013836,0,0.1875,0.013836,1268.061407,1188.749877,0,3,0,0.553762
14179,-1.2,-1.250,0.031855,0,0.1250,0.029871,1242.984723,1188.749877,3,3,0,0.553762
12351,-0.3,-0.375,0.017161,0,0.3750,0.016678,1243.114832,1188.749877,0,3,2,0.553762
18252,0.3,0.250,0.005792,0,0.5625,0.002521,1269.006418,1188.749877,0,3,0,0.553762
...,...,...,...,...,...,...,...,...,...,...,...,...
15699,-3.7,-3.875,0.009170,1,0.0625,0.057972,1724.386078,1353.242229,0,3,0,0.127152
19034,-0.9,-1.000,0.008366,0,0.1875,0.006703,1503.201672,1188.749877,3,3,0,0.553762
6202,-3.7,-3.875,0.003808,0,0.0625,0.003808,1282.017265,1188.749877,0,3,2,0.553762
17094,0.8,0.750,0.002896,0,0.7500,0.002145,1242.984723,1188.749877,0,3,2,0.553762


In [151]:
X_test_mod[features_selected]

,average_rating,number_of_reviews,brand,out_of_stock,avg_delivery_time_days,seller,Style Code,Pack of,Ideal For,Suitable For,Reversible,product
17231,0.2,0.125,0.021719,0,0.5000,0.021719,1279.477181,1188.749877,0,3,0,0.553762
4605,0.3,0.250,0.006114,0,0.5625,0.006060,1279.477181,1353.242229,3,4,2,0.127152
14651,0.2,0.125,0.027511,0,0.5000,0.027082,1190.941333,1188.749877,0,3,2,0.553762
15296,0.2,0.125,0.002467,0,0.5000,0.002413,1210.068432,1188.749877,0,3,0,0.553762
23158,-0.1,-0.125,0.036038,0,0.4375,0.015391,1279.477181,1188.749877,0,3,0,0.093205
...,...,...,...,...,...,...,...,...,...,...,...,...
17710,0.8,0.750,0.014587,0,0.7500,0.014640,1279.477181,1353.242229,3,4,2,0.059688
7684,0.5,0.500,0.033732,0,0.6250,0.055130,1279.477181,1353.242229,3,4,2,0.127152
22683,-0.1,-0.125,0.012495,0,0.4375,0.012495,1279.477181,1656.440327,0,3,0,0.553762
2160,-1.5,-1.625,0.008956,0,0.0625,0.002306,1279.477181,1188.749877,3,3,0,0.553762


In [146]:
def MachineLearning(X_train, y_train, X_test, y_test, features_selected):
    SEED = 42

    lista_de_medidas = ['SMAPE']
    nome_das_medidas = ['SMAPE']

    lista_de_modelos = [XGBRegressor(learning_rate=0.1, n_estimators=100, max_depth=3),
                        SGDRegressor(random_state=SEED), 
                        LGBMRegressor(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        DecisionTreeRegressor(random_state=SEED),
                        KNeighborsRegressor(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingRegressor(),
                        RandomForestRegressor(random_state=SEED)]

    nome_dos_modelos = ['XGBoost', 
                        'SGD', 
                        'LGBM', 
                        'DecisionTree',
                        'KNN',
                        'Bagging',
                        'RandomForest']

    resultados0 = {}
    
    X_train_selected = X_train[features_selected]
    X_test_selected = X_test[features_selected]


    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        
        model = lista_de_modelos[i]
        
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))
        # Calculo meigarom: 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))

        resultados0[nome_dos_modelos[i]] = [smape]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [147]:
MachineLearning(X_train, y_train, X_test_mod, y_test_mod, features_selected)

Rodando modelo: XGBoost
Rodando modelo: SGD


Rodando modelo: LGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 609
[LightGBM] [Info] Number of data points in the train set: 18647, number of used features: 12
[LightGBM] [Info] Start training from score 6.155615
Rodando modelo: DecisionTree
Rodando modelo: KNN
Rodando modelo: Bagging
Rodando modelo: RandomForest


,SMAPE
XGBoost,69.138286
SGD,200.000000
LGBM,70.303567
DecisionTree,71.082815
KNN,69.069873
Bagging,88.117189
RandomForest,88.040976


# Fine Tuning

In [148]:
best_model = XGBRegressor(learning_rate=0.1, n_estimators=100, max_depth=3).fit(X_train[features_selected], y_train)

# Submission

### Carregar os dados

In [149]:
# Opening JSON file
sub = open('dataset/raw/test.json')
 
# returns JSON object as 
# a dictionary
data_sub_aux = json.load(sub)

data_sub = pd.json_normalize(data_sub_aux, record_path = 'data')
data_sub.columns = data_sub_aux['columns']
data_sub.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title
0,35d28961-170f-515d-90d4-55ee978a8afa,4.8,40,Mo,Clothing and Accessories,2021-02-10 21:18:47,Accentuate your formal wardrobe by buying this...,['https://rukminim1.flixcart.com/image/128/128...,False,17,SHTFDYZGEXPED7YS,"[{'Pack of': '1'}, {'Model Name': 'Mods Slim f...",KKSONS,Topwear,32,Men Slim Fit Printed Button Down Collar Casual...
1,5089618f-7eec-571b-84b6-41c3b43a2ed4,3.7,31,True Bl,Clothing and Accessories,2021-02-10 22:12:20,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,10,TROFM47EVWRYK7ZU,"[{'Fit': 'Slim Fit'}, {'Occasion': 'Casual'}, ...",KAPSONSRETAILPVTLTD,Bottomwear,318,Slim Fit Men Brown Cotton Blend Trousers
2,84716375-2ea3-5376-bca1-ecef76b0aa87,3.2,26,vims rai,Clothing and Accessories,2021-02-11 01:16:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,8,FABFZAMPGDW3BKGX,"[{'Type': 'Shirt Fabric'}, {'Sales Package': '...",WHITE SKY,Fabrics,411,Rayon Printed Shirt Fabric (Unstitched)
3,84120873-2b67-5f2c-9074-803e52e278f7,3.6,30,True Bl,Clothing and Accessories,2021-02-10 22:21:32,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,10,BZRFNMGPNBQTYHMG,"[{'Color': 'Blue'}, {'Fabric': 'Polyester'}, {...",KAPSONSRETAILPVTLTD,"Blazers, Waistcoats and Suits",431,Solid Single Breasted Formal Men Full Sleeve B...
4,96238f84-3bb8-5ec1-b2f4-fe481ba35faa,2.5,21,ECKO Unl,Clothing and Accessories,2021-02-10 22:32:00,ECKO Unltd YD CHECK Cotton Woven Slim Fit IND...,['https://rukminim1.flixcart.com/image/128/128...,False,5,SHTFV5G6PZDQZEVZ,"[{'Pack of': '1'}, {'Style Code': 'EKSH001497'...",SandSMarketing,Topwear,512,Men Slim Fit Checkered Cut Away Collar Casual ...


### Aplicar ML no teste e submissão

In [150]:
# Aplicando funções nos dados de teste
data_sub_mod = submission_preparation(data_sub, features_selected)

# Aplicando o modelo nos dados de teste
y_hat = np.expm1(best_model.predict(data_sub_mod))

ValueError: Unexpected input dimension 13, expected 24

In [ ]:
# Preparando os dados para a submissão

df_submission = pd.DataFrame()
df_submission['pid'] = data_sub['pid']
df_submission['actual_price'] = y_hat

# Exportando como CSV

df_submission.to_csv('submissions/submission_1.csv', index=False)